In [1]:
from models import *
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import Compose, ToTensor, Resize, CenterCrop
from torchmetrics import AUROC
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import timm


In [2]:
device = torch.device("cuda:0") 

In [3]:
###Modify Me according to your paths###
ckpt_dir = "/home/SHARED_FOLDER/checkpoints/"
ood_data_dir = "/home/SHARED_FOLDER/data/OOD_data/"
id_data_dir = "/home/SHARED_FOLDER/data/ID_data/imagenet1k-val/"

# prepare data

500 ID images and 500 OOD images
target for ID images is 1 and target for OOD images is 0. (binary OOD detection)

In [4]:
ori_preprocess = Compose([
        Resize((224), interpolation=Image.BICUBIC),
        CenterCrop(size=(224, 224)),
        ToTensor()])

/opt/conda/lib/python3.7/site-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [7]:
class Image_Dataset(Dataset):
    def __init__(self, id_data_dir, ood_data_dir):
        self.id_data_dir = id_data_dir
        self.ood_data_dir = ood_data_dir
        self.data = []
        for file in os.listdir(self.id_data_dir)[:1]:
            self.data.append((self.id_data_dir+file, 0))
        for file in os.listdir(self.ood_data_dir)[:1]:
            self.data.append((self.ood_data_dir+file), 1)
        print(self.data)

    def __getitem__(self, idx):
        
        img_path, target = self.data[idx]
        img = ori_preprocess(Image.open(img_path))

        print(img)
        print(target)

        return img, target

    def __len__(self):
        return len(self.data)

In [8]:
dataset = Image_Dataset(id_data_dir, ood_data_dir)
data_loader = DataLoader(dataset, batch_size = 1, shuffle= True)

[('/home/SHARED_FOLDER/data/ID_data/imagenet1k-val/ILSVRC2010_val_00019553.JPEG', 0)]


In [7]:
def precision(model, dataloader, thresholds, device, num_classes=1000, batch_size=1):

    probs = torch.zeros((batch_size, num_classes))
    targets = torch.zeros((batch_size, 1))
    model.to(device)
    model.eval()
    # check value for threshold
    prc = Precision(num_classes = num_classes, threshold=1)
    with torch.no_grad():
        for ind, input, batch_target in enumerate(dataloader):
            input.to(device)
            
            batch_probs = model(input)
            if ind==0:
                probs = batch_probs
                targets = batch_target
            else:
                # check
                print(probs.size())
                probs = torch.concat((probs, batch_probs), dim=1)
                print(probs.size())
                targets = torch.concat((targets, batch_target), dim=1)
        
    precisions = torch.zeros((len(thresholds),1))
    for ind,t in enumerate(thresholds):
        # check
        preds = torch.max(probs, dim=1) < t
        precision[ind] = prc(preds, targets)


    return precisions

# ResNet

In [ ]:
ood_threshold = 0.5
resnet_model = resnet.ResNet(f"{ckpt_dir}resnet50_a2_0-a2746f79.pth", ood_threshold).to(device)
# the goal is to identify OOD samples

thresholds = torch.range(0.9, step=0.1)
probs = torch.zeros((batch_size, num_classes))
targets = torch.zeros((batch_size, 1))
model.to(device)
model.eval()
# check value for threshold
# prc = Precision(num_classes = num_classes, threshold=1)

In [8]:

with torch.no_grad():
    for ind, input_, batch_target in enumerate(dataloader):
        input_.to(device)
        batch_probs = model(input_)
        if ind==0:
            probs = batch_probs
            targets = batch_target
        else:
            # check
            print(probs.size())
            probs = torch.concat((probs, batch_probs), dim=1)
            print(probs.size())
            targets = torch.concat((targets, batch_target), dim=1)
    


SyntaxError: invalid syntax (2725935056.py, line 5)

In [ ]:
precisions = torch.zeros((len(thresholds),1))

for ind,t in enumerate(thresholds):
    # check
    preds = torch.max(probs, dim=1) < t
    precision[ind] = prc(preds, targets)


#resnet_precisions = precision(resnet_model, dataloader, thresholds, device)
# tp = 0
# resnet_preds = torch.zeros((8,1))
# for data, target in dataloader:
#     data.to(device)
#     resnet_preds = resnet_model(data)
    
#     for i, pred in enumerate(resnet_preds):
#         if pred == target[i]: tp+=1

# precision = tp / len(dataset)
# print()


In [ ]:
resnet_auroc = eval_metric(resnet_preds, targets)
print(resnet_auroc)

/home/mamooler/anaconda3/envs/sem_proj/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


tensor(0.0008)


# DeiT

In [ ]:
deit_model = deit.DeiT(f"deit_small_patch16_a2_0-83b53863.pth")
deit_preds = deit_model(data)


Using cache found in /home/mamooler/.cache/torch/hub/facebookresearch_deit_main


In [ ]:
deit_auroc = eval_metric(deit_preds, targets)
print(deit_auroc)

Error: Kernel is dead

# ConvMixer

In [ ]:
convmixer_model = convmixer.ConvMixer()
convmixer_preds = convmixer_model(data)


In [ ]:
convMixer_auroc = eval_metric(convmixer_preds, targets)
print(convMixer_auroc)

# MLPMixer

In [ ]:
mlpmixer_model = mlpmixer.MLPMixer()
mlpmixer_preds = mlpmixer_model(data)


In [ ]:
mlpmixer_auroc = eval_metric(mlpmixer_preds, targets)
print(mlpmixer_auroc)

# ECA-ResNet

In [ ]:
ecaresnet_model = ecaresnet.ECAResNet()
ecaresnet_preds = ecaresnet_model.ecaresnet(data)

In [ ]:
ecaresnet_auroc = eval_metric(ecaresnet_preds, targets)
print(ecaresnet_auroc)